<a href="https://colab.research.google.com/github/RamziRBM/lab-chains-in-langchain/blob/main/lab-chains-in-langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import userdata
import os

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
# Set it as an environment variable, which LangChain often uses by default
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
print("OpenAI API key loaded and set as environment variable.")

HUGGINGFACEHUB_API_TOKEN= userdata.get('HUGGINGFACEHUB_API_TOKEN')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN
print("HUGGINGFACEHUB API key loaded and set as environment variable.")

OpenAI API key loaded and set as environment variable.
HUGGINGFACEHUB API key loaded and set as environment variable.


In [ ]:
#!pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv('https://github.com/ironhack-labs/lab-chains-in-langchain/raw/refs/heads/main/data/Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [21]:
!pip install langchain langchain-openai python-dotenv

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.7)

print("Model being used:", llm.model_name)



Model being used: gpt-3.5-turbo


In [27]:
!pip install langchain_classic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.7 MB/s eta 0:00:00


In [28]:
from langchain_openai import ChatOpenAI
#from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.llm import LLMChain

In [29]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.7)


In [30]:
prompt = ChatPromptTemplate.from_template( #Write a query that would take a variable to describe any product
 """Write a detailed and engaging review about the product: {product}.
    Highlight its key features, quality, sound, and why someone might want to buy it."""
)

In [31]:

chain = LLMChain(llm=llm, prompt=prompt)

/tmp/ipython-input-546483037.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [33]:
product = 'Pillows Insert'#Select a product type to be describe
chain.run(product)

/tmp/ipython-input-4029614091.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  chain.run(product)


"I recently purchased a set of Pillows Inserts and I have been absolutely thrilled with my purchase. These pillow inserts have completely transformed the look and feel of my living room, and I couldn't be happier with the quality of the product.\n\nOne of the key features of these pillow inserts is their exceptional quality. The inserts are made from high-quality materials that are durable and long-lasting. The stitching is also incredibly well done, ensuring that the inserts will hold up well over time. The inserts are also incredibly soft and plush, providing the perfect amount of support for a comfortable night's sleep or lounging on the couch.\n\nAnother standout feature of these pillow inserts is their versatility. They come in a variety of sizes, making it easy to find the perfect fit for any pillow cover. Whether you have a standard size pillow or a larger decorative pillow, these inserts are sure to fit seamlessly. The inserts are also incredibly easy to care for, as they are m

## SimpleSequentialChain

In [34]:
from langchain_classic.chains import SimpleSequentialChain

In [35]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY,
        model="gpt-3.5-turbo", temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "As an expert in product marketing, analyze the following product description:\n\n{product_description}\n\n"
    "Based on this, suggest 3 concise, catchy, and benefit-oriented taglines. "
    "Format them as a numbered list."
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [36]:

second_prompt = ChatPromptTemplate.from_template(
    "Based on these taglines for a product:\n\n{taglines_from_first_chain}\n\n"
    "Suggest a creative, short social media post (e.g., for Instagram or X) "
    "that incorporates one of these taglines. Include relevant emojis and hashtags."
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [37]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [38]:
product = "A revolutionary smart thermostat that learns your preferences, saves energy, and can be controlled remotely via a smartphone app. It features a sleek design and easy installation."

overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
1. "Effortless comfort at your fingertips."
2. "Save energy, stay cozy."
3. "Smart control, stylish design."
"Stay cozy all winter long with our smart control heating system! 🌡️✨ #EffortlessComfort #SaveEnergy #SmartDesign"

> Finished chain.


'"Stay cozy all winter long with our smart control heating system! 🌡️✨ #EffortlessComfort #SaveEnergy #SmartDesign"'

In [39]:
overall_simple_chain.invoke(df["Review"][3])



> Entering new SimpleSequentialChain chain...
1. Elevate your comfort with our premium throw pillow fillers
2. Say goodbye to flat pillows with our fluff-tastic fillers
3. Experience the ultimate fluffiness with our dryer-friendly pillow fillers
"Upgrade your pillow game with our premium fillers! 🌟 Say goodbye to flat pillows and hello to ultimate fluffiness with our dryer-friendly fillers! 💫 #PillowPerfection #FluffTastic"

> Finished chain.


{'input': 'This is the best throw pillow fillers on Amazon. I’ve tried several others, and they’re all cheap and flat no matter how much fluffing you do. Once you toss these in the dryer after you remove them from the vacuum sealed shipping material, they fluff up great',
 'output': '"Upgrade your pillow game with our premium fillers! 🌟 Say goodbye to flat pillows and hello to ultimate fluffiness with our dryer-friendly fillers! 💫 #PillowPerfection #FluffTastic"'}

In [40]:
overall_simple_chain(df["Review"][4])

/tmp/ipython-input-3683474712.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  overall_simple_chain(df["Review"][4])




> Entering new SimpleSequentialChain chain...
1. Upgrade your morning coffee routine with our durable frother replacement policy. 
2. Say goodbye to frother frustrations and hello to quality that lasts. 
3. Experience the convenience and reliability of our long-lasting frother design.
"Upgrade your coffee game with our durable frother replacement policy ☕️ Say goodbye to frother frustrations and enjoy quality that lasts! #CoffeeLovers #FrotherUpgrade"

> Finished chain.


{'input': "\xa0I loved this product. But they only seem to last a few months. The company was great replacing the first one (the frother falls out of the handle and can't be fixed). The after 4 months my second one did the same. I only use the frother for coffee once a day. It's not overuse or abuse. I'm very disappointed and will look for another. As I understand they will only replace once. Anyway, if you have one good luck.",
 'output': '"Upgrade your coffee game with our durable frother replacement policy ☕️ Say goodbye to frother frustrations and enjoy quality that lasts! #CoffeeLovers #FrotherUpgrade"'}

**Repeat the above twice for different products**

## SequentialChain

In [46]:
from langchain_classic.chains import SequentialChain

In [41]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY,
        model="gpt-3.5-turbo", temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
    "Translate the following product review into {target_language}:\n\nReview: \"{review_text}\""
)

chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="english_review_output" # This matches 'english_review_output' in overall_chain's output_variables
                    )



In [42]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following product review in 1-2 concise sentences:\n\nReview: \"{review_text}\""
)
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="review_summary" # This matches 'review_summary' in overall_chain's output_variables
                    )

In [43]:
# prompt template 3: translate to english or other language

third_prompt = ChatPromptTemplate.from_template(
    "Translate the following summary into {target_language}:\n\nSummary: \"{review_summary}\"" # Uses output from chain_two
)
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="translated_summary_output" # Fixed typo and named clearly
                      )

In [44]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
    "Given the original product review:\n\"\"\"{review_text}\"\"\"\n\n" # Using {review_text} as the overall input
    "And its translated summary:\n\"\"\"{translated_summary_output}\"\"\"\n\n" # Using output from chain_three
    "Write a concise, empathetic, and professional follow-up message to the customer regarding their review. "
    "Offer a solution or next step if appropriate."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                       output_key="follow_up_message" # This matches 'follow_up_message' in overall_chain's output_variables
                      )


In [47]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review_text", "target_language"], # <-- This is the fix for your ValidationError!
    output_variables=["english_review_output", "review_summary", "follow_up_message"],
    verbose=True
)

In [49]:
review_text_to_process = df.Review[2]
target_language_for_translation = "French" # You specified Gujarati in your invoke call

# Correct way to invoke the chain:
# Pass a single dictionary with keys matching overall_chain's input_variables
results = overall_chain.invoke({
    "review_text": review_text_to_process,
    "target_language": target_language_for_translation
})

print("\n--- Processed Output (Line by Line) ---")
for key, value in results.items():
    print(f"{key.replace('_', ' ').title()}:") # Makes keys look nicer (e.g., "Review Text")
    print(f"  {value}\n") # Indent the value and add a blank line for separation



> Entering new SequentialChain chain...

> Finished chain.

--- Processed Output (Line by Line) ---
Review Text:
  This mattress had a small hole in the top of it (took forever to find where it was), and the patches that they provide did not work, maybe because it's the top of the mattress where it's kind of like fabric and a patch won't stick. Maybe I got unlucky with a defective mattress, but where's quality assurance for this company? That flat out should not happen. Emphasis on flat. Cause that's what the mattress was. Seriously horrible experience, ruined my friend's stay with me. Then they make you ship it back instead of just providing a refund, which is also super annoying to pack up an air mattress and take it to the UPS store. This company is the worst, and this mattress is the worst.

Target Language:
  French

English Review Output:
  Avis : Ce matelas avait un petit trou sur le dessus (il a fallu une éternité pour le trouver), et les rustines fournies ne fonctionnaient p

**Repeat the above twice for different products or reviews**

## Router Chain

In [50]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [51]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [52]:
from langchain_classic.chains.router import MultiPromptChain
from langchain_classic.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain_classic.prompts import PromptTemplate

In [53]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY,
        model="gpt-3.5-turbo", temperature=0.6)

In [54]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [56]:
type(destinations_str), destination_chains[name]

(str,
 LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an excellent biologist. You have a deep understanding of living organisms, from the molecular and cellular level to entire ecosystems. You are skilled at observing patterns in nature, analyzing biological data, and explaining complex processes like evolution, genetics, physiology, and ecology. You can clearly communicate how life functions and adapts, and you make connections between different biological concepts to answer challenging questions.\n\nHere is a question:\n{input}'), additional_kwargs={})]), llm=ChatOpenAI(profile={'max_input_tokens': 16385, 'max_output_tokens': 4096, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reason

In [57]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [58]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [59]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [60]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

/tmp/ipython-input-3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [61]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect absorber and emitter of radiation, known as a black body. A black body absorbs all radiation that falls on it and emits radiation over a wide range of frequencies. The intensity and distribution of this radiation is determined by its temperature, with hotter objects emitting more radiation at shorter wavelengths.\n\nThis phenomenon is described by Planck's law, which states that the intensity of radiation emitted by a black body is a function of its temperature and the wavelength of the radiation. Black body radiation plays a key role in understanding the behavior of objects at high temperatures, such as stars and incandescent light bulbs."

In [62]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'2 + 2 is equal to 4.'

In [63]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the molecule that carries the genetic information necessary for the growth, development, and function of all living organisms. DNA contains the instructions for making proteins, which are essential for the structure and function of cells. \n\nAdditionally, DNA is responsible for passing on genetic traits from one generation to the next through the process of reproduction. Having DNA in every cell ensures that each cell has the information it needs to carry out its specific functions and to replicate accurately when the cell divides.\n\nOverall, the presence of DNA in every cell is crucial for the proper functioning and survival of an organism.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**